In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install -U datasets


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from evaluate import load

In [ ]:
df=pd.read_csv('/content/Dataset - Sheet1.csv')

In [ ]:
df = pd.read_csv('/content/Dataset - Sheet1.csv')
print(df.head())

   Unnamed: 0 Unnamed: 1                  Unnamed: 2  \
0         NaN        pdf                       tasks   
1         NaN       305a  complete assigned readings   
2         NaN        NaN           prepare for exams   
3         NaN        NaN       submit optional paper   
4         NaN     PSY345                  term tests   

                     Unnamed: 3  
0                class assigned  
1                 Participation  
2                         Exams  
3  Designs,Reports and Analyses  
4                         Exams  


In [ ]:
print("Columns in the DataFrame:", df.columns)


Columns in the DataFrame: Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'], dtype='object')


In [ ]:
df = df.drop(columns=['Unnamed: 0'], errors='ignore') #remove the first unnamed column
df = df.drop(0).reset_index(drop=True) #remove first row
df.columns = ['pdf', 'tasks', 'class']
df['pdf'] = df['pdf'].fillna(method='ffill')

print(df.head(10))

      pdf                       tasks                         class
0    305a  complete assigned readings                 Participation
1    305a           prepare for exams                         Exams
2    305a       submit optional paper  Designs,Reports and Analyses
3  PSY345                  term tests                         Exams
4  PSY345                     quizzes                   Assignments
5  PSY345            paper submission  Designs,Reports and Analyses
6  PSY100                        labs                   Assignments
7  PSY100                  term tests                         Exams
8  MAT392   assignment1: essay draft,  Designs,Reports and Analyses
9  MAT392    assignment2: first essay  Designs,Reports and Analyses


<ipython-input-11-bb0bb3e65307>:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['pdf'] = df['pdf'].fillna(method='ffill')


In [ ]:
df = df.dropna(subset=['tasks', 'class'])  # Ensure no missing values in relevant columns


In [ ]:
dataset = Dataset.from_pandas(df)
dataset = dataset.class_encode_column("class")



Casting to class labels:   0%|          | 0/891 [00:00<?, ? examples/s]

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["tasks"], padding="max_length", truncation=True)

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/891 [00:00<?, ? examples/s]

In [ ]:
df['class']

,class
0,Participation
1,Exams
2,"Designs,Reports and Analyses"
3,Exams
4,Assignments
...,...
890,Participation
891,"Designs,Reports and Analyses"
892,Exams
893,Assignments


In [ ]:

tokenized_datasets = tokenized_datasets.map(lambda x: {'labels': x['class']})


Map:   0%|          | 0/891 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_datasets)

Dataset({
    features: ['pdf', 'tasks', 'class', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 891
})


In [ ]:

train_val_split = tokenized_datasets.train_test_split(test_size=0.2)
test_valid_split = train_val_split['test'].train_test_split(test_size=0.5)

In [ ]:
tokenized_datasets = {
    'train': train_val_split['train'],
    'test': test_valid_split['test'],
    'validation': test_valid_split['train']
}

In [ ]:

accuracy_metric = load("accuracy")
precision_metric = load("precision")
f1_metric = load("f1")


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    accuracy = accuracy_metric.compute(predictions=preds, references=labels)["accuracy"]
    precision = precision_metric.compute(predictions=preds, references=labels, average="weighted")["precision"]
    f1 = f1_metric.compute(predictions=preds, references=labels, average="weighted")["f1"]

    return {
        "accuracy": accuracy,
        "precision": precision,
        "f1": f1
    }

In [ ]:
num_labels = dataset.features["class"].num_classes
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=num_labels)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(num_labels)

8


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision,F1
1,No log,1.153671,0.606742,0.425000,0.488275
2,No log,0.869678,0.741573,0.755935,0.703075
3,No log,0.747714,0.786517,0.792701,0.761398


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=267, training_loss=1.0007046474499648, metrics={'train_runtime': 15546.1451, 'train_samples_per_second': 0.137, 'train_steps_per_second': 0.017, 'total_flos': 562035490357248.0, 'train_loss': 1.0007046474499648, 'epoch': 3.0})

In [ ]:

test_results = trainer.evaluate(tokenized_datasets['test'])

print("Test Set Results:")
print(f"Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"Precision: {test_results['eval_precision']:.4f}")
print(f"F1 Score: {test_results['eval_f1']:.4f}")


Test Set Results:
Accuracy: 0.8333
Precision: 0.8597
F1 Score: 0.8028


##This part is about a single .txt

In [ ]:
tasks=extract_tasks_with_percentages("/content/test1.txt")
print(tasks)

In [ ]:
# Tokenize tasks for input into the model
def tokenize_tasks(tasks, tokenizer, max_length=128):
    return tokenizer(tasks, padding=True, truncation=True, max_length=max_length, return_tensors="pt")

# Tokenize the extracted tasks
inputs = tokenize_tasks(tasks, tokenizer)


In [ ]:
class_labels = dataset.features["class"].names  # Assuming labels were saved in the dataset
print("Class labels:", class_labels)


In [ ]:
import torch


In [ ]:
import torch

# Ensure consistent device usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move the model to the device
model = model.to(device)

# Move tokenized inputs to the same device
inputs = {key: value.to(device) for key, value in inputs.items()}

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)




In [ ]:
# Use the trained RoBERTa model for predictions
def classify_tasks_with_roberta(inputs, model):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)  # Get the predicted class indices
    return predictions

# Classify tasks
predictions = classify_tasks_with_roberta(inputs, model)


In [ ]:
predicted_labels = [class_labels[pred] for pred in predictions]


In [ ]:
import pandas as pd

# Combine extracted tasks and predicted labels
results = pd.DataFrame({
    "Task": tasks,
    "Predicted Category": predicted_labels
})

# Display the results
print(results)


##Scriot to extract tasks from a bunch of notebooks.

In [ ]:
import pandas as pd

# Load the data file (assuming it's in a delimited text format)
# Update the file path and delimiter as needed
file_path = "/content/test1.txt"
data = pd.read_csv(file_path, delimiter='|')

# Clean up the column headers and data (if necessary)
data.columns = data.columns.str.strip()  # Strip whitespace from column names
data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Strip whitespace from data

# Extract the "Task" column
task_column = data["Task"]




# Output the "Task" column
print(task_column)

# Optionally save to a new file
task_column.to_csv("extracted_tasks.csv", index=False)


0                               ---
1                       Assignments
2                     Mid-Term Test
3                   Final Term Test
4    Class Participation/Attendance
5         Reading Required Textbook
6         Reading Optional Textbook
7            UCheck Self-Assessment
8                Recording Lectures
Name: Task, dtype: object


<ipython-input-31-b2f71420a697>:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Strip whitespace from data


In [ ]:
import pandas as pd

# Update the file path and delimiter as needed
file_path = "/content/test1.txt"  # Update this with the correct path
data = pd.read_csv(file_path, delimiter='|')

# Clean up the column headers and data (if necessary)
data.columns = data.columns.str.strip()  # Strip whitespace from column headers
data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Strip whitespace from string values

# Remove rows where the 'Task' column contains '---'
data = data[data["Task"] != "---"]

# Extract the "Task" column
task_column = data["Task"]

# Verify the content of task_column
print(task_column)  # Should only contain the "Task" column data

# Save only the "Task" column to a new CSV
task_column.to_csv("cleaned_task_column.csv", index=False, header=True)
print("Cleaned task column saved to 'cleaned_task_column.csv'.")



1                       Assignments
2                     Mid-Term Test
3                   Final Term Test
4    Class Participation/Attendance
5         Reading Required Textbook
6         Reading Optional Textbook
7            UCheck Self-Assessment
8                Recording Lectures
Name: Task, dtype: object
Cleaned task column saved to 'cleaned_task_column.csv'.


<ipython-input-32-d4b398833343>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Strip whitespace from string values


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
file_path = "/content/cleaned_task_column.csv"  # Update with your actual file path
data = pd.read_csv(file_path)
task_list = data["Task"].tolist()  # Convert the "Task" column to a Python list

In [ ]:
print(task_list)

['Assignments', 'Mid-Term Test', 'Final Term Test', 'Class Participation/Attendance', 'Reading Required Textbook', 'Reading Optional Textbook', 'UCheck Self-Assessment', 'Recording Lectures']


In [ ]:
tokenized_inputs = tokenizer(task_list, padding=True, truncation=True, return_tensors="pt")


In [ ]:
print(tokenized_inputs)

{'input_ids': tensor([[    0, 26039,  4932,  2963,     2,     1,     1,     1],
        [    0, 30238,    12, 23036,  4500,     2,     1,     1],
        [    0, 38083, 25569,  4500,     2,     1,     1,     1],
        [    0, 21527, 39332,    73, 28062,  1397,  2389,     2],
        [    0, 43952, 41705, 14159,  6298,     2,     1,     1],
        [    0, 43952, 44526, 14159,  6298,     2,     1,     1],
        [    0,   791, 26615, 12156,    12, 26039, 38170,     2],
        [    0, 21109, 36473, 29893,  4123,     2,     1,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0]])}


In [ ]:
from torch.utils.data import Dataset

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, tokenized_data):
        self.input_ids = tokenized_data["input_ids"]
        self.attention_mask = tokenized_data["attention_mask"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
        }

# Wrap tokenized_inputs in the CustomDataset
test_dataset = CustomDataset(tokenized_inputs)

# Pass the dataset to trainer.predict()
predictions = trainer.predict(test_dataset)
predicted_classes = predictions.predictions.argmax(axis=-1)

# Add predictions to your datafr

print(predicted_classes)

[7 2 2 5 1 1 1 6]
